# Assignment 3 Advanced programming

Script to analyse molecule ALDL1 inhibition.

In [29]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from rdkit.Chem import PandasTools, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn import pipeline as pl
from sklearn import preprocessing as pp
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
#import graphviz 

Location of data

In [30]:
PATH_TESTED_MOLECULES =     ['data\\tested_molecules_v2.csv']
PATH_UNTESTED_MOLECULES =   'data\\untested_molecules.csv'

# 1 Load tested molecules

We received two files with tested molecules for ALDH1 inhibition. Analysis showed that the prediction algorithms worked with sensitivity, specifity and precision when only the v2 dataset was used! The first dataset showed mediocre results. We decided, though tempting to use only v2, to uses both sets for the predictions, since we don't have any guarantee that the unknown molecules behave more like v2-molecules.

In [31]:
dataset_molecules = pd.DataFrame()
for i, file in enumerate(PATH_TESTED_MOLECULES):
    tested_molecules = pd.read_csv(os.path.join(os.getcwd(), file))
    PandasTools.AddMoleculeColumnToFrame(tested_molecules, smilesCol='SMILES')
    tested_molecules['FROM DATASET'] = i
    dataset_molecules = dataset_molecules.append(tested_molecules, ignore_index = True)
    
dataset_molecules.head()

,SMILES,ALDH1_inhibition,ROMol,FROM DATASET
0,[NH3+]CCSSCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x0000025B32F...,0
1,[NH3+]CCC[NH2+]CCCC[NH2+]CCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x0000025B32F...,0
2,[NH3+]CCCCCCCCCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x0000025B32F...,0
3,[NH3+]CCSSCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x0000025B32F...,0
4,ClCC[NH+](CCCl)CCCl,0,<rdkit.Chem.rdchem.Mol object at 0x0000025B32F...,0


# 2 Preparation of molecular descriptors

Add all molecular descriptor and calculate them to a pd DataFrame dataset_descriptors

In [32]:
descriptors_labels = [n[0] for n in Descriptors._descList[:]]
calc = MoleculeDescriptors.MolecularDescriptorCalculator(descriptors_labels)
rdkit_desc = [calc.CalcDescriptors(m) for m in dataset_molecules["ROMol"]]
dataset_descriptors = pd.DataFrame(rdkit_desc, columns=[descriptors_labels])

Select only the 'good' descriptors, as calculated by MART........

In [33]:
dataset_descriptors = dataset_descriptors[['MaxAbsEStateIndex',
 'MinAbsEStateIndex',
 'MinEStateIndex',
 'BCUT2D_CHGHI',
 'BCUT2D_CHGLO',
 'BCUT2D_LOGPLOW',
 'BCUT2D_MRLOW',
 'SMR_VSA1',
 'SMR_VSA5',
 'SlogP_VSA2',
 'EState_VSA10',
 'VSA_EState2',
 'FractionCSP3',
 'NumAliphaticRings']]

dataset_descriptors.head()

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPLOW,BCUT2D_MRLOW,SMR_VSA1,SMR_VSA5,SlogP_VSA2,EState_VSA10,VSA_EState2,FractionCSP3,NumAliphaticRings
0,3.733529,1.036517,1.036517,1.865112,-1.907393,-2.514714,-0.360469,11.467335,0.000000,24.595220,0.0,0.000000,1.0,0
1,3.831341,1.078701,1.078701,1.940948,-2.003261,-2.860321,-0.666457,22.100912,25.683286,39.268538,0.0,0.000000,1.0,0
2,3.837048,1.112603,1.112603,1.908706,-1.984755,-2.495713,-0.368558,11.467335,51.366573,13.089513,0.0,0.000000,1.0,0
3,3.733529,1.036517,1.036517,1.865112,-1.907393,-2.514714,-0.360469,11.467335,0.000000,24.595220,0.0,0.000000,1.0,0
4,5.560741,0.683642,0.683642,2.035557,-2.143624,-3.126526,-0.892277,4.899910,0.000000,37.274280,0.0,1.388889,1.0,0


# 3 Do the trick on the known molecules

### Divide the known dataset in test and train

In [34]:
X_train, X_test, y_train, y_test = train_test_split(dataset_descriptors, dataset_molecules['ALDH1_inhibition'], test_size=0.2, random_state=42)

### Introducing a Pipeline with minmax scaler and Desciscion tree.

In [35]:
scaler = pp.MinMaxScaler()
pca = PCA()
dtree = tree.DecisionTreeClassifier(max_features=None, max_depth=None, max_leaf_nodes=None)

pipeline = pl.Pipeline(steps=[("sc", scaler), ("pca", pca), ("dtreeCLF", dtree)])
cross_score = cross_val_score(pipeline, X_train, y_train, cv=10)
print("Cross score:") 
print(cross_score)
pipeline.fit(X_train, y_train)
print("score:")
print(pipeline.score(X_test, y_test))

Cross score:
[0.9375 0.975  0.975  0.9625 0.975  0.9875 0.95   0.9625 0.95   0.9625]
score:
0.96


### Creating a physical tree

In [36]:
#dot_data = tree.export_graphviz(dtree, out_file=None, feature_names=dataset_descriptors.columns, class_names=['1', '0'])
#graph = graphviz.Source(dot_data) 
#graph.render("descision_tree") 

### Analysing the model

In [37]:
y_true = y_test
y_pred = pipeline.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp/(tp+fp) 
accuracy = (tp+tn)/(tp+tn+fp+fn)
tpr = tp/(tp+fp)
tnr = tn/(tn+fn)
balanced_accuracy = (tpr+tnr)/2
print(" |+\t|-\t|\n+|{:.0f}\t|{:.0f}\t|\n-|{:.0f}\t|{:.0f}\t|".format(tp, fp, fn, tn))
print("Specificiteit: {:.2f} Sensitivity: {:.2f} Accuracy {:.2f}, Balanced Accuracy: {:.2f}, TPR: {:.2f}".format(specificity, sensitivity, accuracy, balanced_accuracy, tpr))

 |+	|-	|
+|51	|2	|
-|6	|141	|
Specificiteit: 0.99 Sensitivity: 0.96 Accuracy 0.96, Balanced Accuracy: 0.96, TPR: 0.96


## Further processing of the selection

Create new datafame 'pipeline_results' from X_test. Append values for 'ALDH1_inhibition' and 'predicted'

In [38]:
pipeline_results = X_test.copy()
pipeline_results['ALDH1_inhibition'] = y_test
pipeline_results['predicted'] = y_pred
pipeline_results.head()

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPLOW,BCUT2D_MRLOW,SMR_VSA1,SMR_VSA5,SlogP_VSA2,EState_VSA10,VSA_EState2,FractionCSP3,NumAliphaticRings,ALDH1_inhibition,predicted
521,6.025203,0.013334,0.013334,2.123551,-2.058758,-2.040441,0.999525,0.000000,0.000000,15.181342,0.000000,3.800942,0.000000,0,0,0
737,12.228184,0.026300,-3.984782,2.208004,-2.258658,-2.374459,-0.385183,18.135645,31.584601,42.949395,23.326652,25.091852,0.428571,1,1,1
740,12.276627,0.171944,-0.171944,2.243063,-2.090296,-2.207212,0.095441,4.794537,19.765380,16.185964,4.794537,12.276627,0.250000,1,1,0
660,4.116019,0.871623,0.871623,2.048714,-2.021316,-1.924383,0.900490,0.000000,0.000000,15.410749,0.000000,0.000000,0.000000,0,0,0
411,6.010897,0.381960,0.381960,2.019603,-2.062337,-1.969226,0.293773,4.736863,6.606882,4.983979,0.000000,4.049655,0.083333,0,0,0


Select only the predicted positives

In [39]:
pipeline_positives = pipeline_results[pipeline_results['predicted'].values == 1]
print("Result: {} positives were selected from pipeline".format(pipeline_positives.shape[0]))

Result: 53 positives were selected from pipeline


In [40]:
type(pipeline_positives)

pandas.core.frame.DataFrame

## Further Selection with Tanamoto similarity

Load all known molecules in frame.

In [41]:
from rdkit import DataStructs
tested_molecules = dataset_molecules.copy()
PandasTools.AddMoleculeColumnToFrame(tested_molecules, smilesCol='SMILES')

Divide molecules in Inhibitors and non-Inhibitors

In [42]:
tested_molecules_pos = tested_molecules[tested_molecules['ALDH1_inhibition'] == 1]
tested_molecules_neg = tested_molecules[tested_molecules['ALDH1_inhibition'] == 0]
print("{}% inhibitors present in known molecules".format(100*len(tested_molecules_pos)/len(tested_molecules)))

30.0% inhibitors present in known molecules


Convert both sets to Morgan Fingerprints (to compare with)

In [43]:
from rdkit.Chem import AllChem
bulk_pos = [AllChem.GetMorganFingerprintAsBitVect(x,2) for x in tested_molecules_pos['ROMol']]
bulk_neg = [AllChem.GetMorganFingerprintAsBitVect(x,2) for x in tested_molecules_neg['ROMol']]
len_pos = len(bulk_pos); len_neg = len(bulk_neg)

Get indices from pipeline prediction and select these indices from dataset_molecules.
This must be done to retreive SMILES again, since these are needed for the comparison in the for loop

In [44]:
index_pipeline_positives = pipeline_positives.index
pipeline_positives_mol = dataset_molecules.iloc[index_pipeline_positives]
print("{} possible inhibitors were seclected from pipeline".format(len(pipeline_positives_mol)))

53 possible inhibitors were seclected from pipeline


Calculate similarity with tested positives/negatives for every mol from pipeline_positives. Add the similarity to columns.

In [45]:
pipeline_positives_mol_copy = pipeline_positives_mol.copy(deep=False)
# Loop all rows from test set
for index, row in pipeline_positives_mol.iterrows():     
    test_smiles = row['SMILES']
    # Make allchem MolFrom the SMILES and convert it into to GetMorganFingerprintAsBitVec
    test_mol = AllChem.MolFromSmiles(test_smiles)
    ref_fps = AllChem.GetMorganFingerprintAsBitVect(test_mol,2)
    # Calculate similarities with positives and negative
    similarity_pos = [DataStructs.FingerprintSimilarity(ref_fps,x) for x in bulk_pos]
    similarity_neg = [DataStructs.FingerprintSimilarity(ref_fps,x) for x in bulk_neg]
    # Add all positive and negative similarities and normalize
    rel_sum_pos = sum(similarity_pos)/len_pos # divide by sizes to normalize the result
    rel_sum_neg = sum(similarity_neg)/len_neg # divide by sizes to normalize the result
    
    pipeline_positives_mol_copy.loc[index, ['SIM_WITH_POSITIVE_SET']] = rel_sum_pos
    pipeline_positives_mol_copy.loc[index, ['SIM_WITH_NEGATIVE_SET']] = rel_sum_neg   

pipeline_positives_mol_copy.head()

,SMILES,ALDH1_inhibition,ROMol,FROM DATASET,SIM_WITH_POSITIVE_SET,SIM_WITH_NEGATIVE_SET
737,CC(=O)N1CCC(=NNS(=O)(=O)c2ccc(C)c([N+](=O)[O-]...,1,<rdkit.Chem.rdchem.Mol object at 0x0000025B330...,0,0.158770,0.053253
859,CC(=O)c1cccc(NC(=O)N2CCCC2C(=O)Nc2c(C)cccc2C)c1,1,<rdkit.Chem.rdchem.Mol object at 0x0000025B330...,0,0.166925,0.067149
811,Cc1ccc(/N=C/c2cc(C)n(C3CCCC3)c2C)cc1,1,<rdkit.Chem.rdchem.Mol object at 0x0000025B330...,0,0.125908,0.055668
973,O=C([O-])c1ccccc1SC1CC(=O)N(C2CCCCC2)C1=O,1,<rdkit.Chem.rdchem.Mol object at 0x0000025B330...,0,0.150318,0.073194
938,COC(=O)c1c(-c2ccco2)csc1NC(=O)C1CC=CCC1C(=O)[O-],1,<rdkit.Chem.rdchem.Mol object at 0x0000025B330...,0,0.145305,0.064934


Since we know the real ALDH1_inibition, we can calculate the number of false positives from the prediction

In [46]:
false_positives = pipeline_positives_mol_copy[pipeline_positives_mol_copy['ALDH1_inhibition'] == 0]
print("{:d} predicted positives from pipeline. {:.1f}% false. ".format(len(pipeline_positives_mol_copy),100*len(false_positives)/len(pipeline_positives_mol_copy)))

53 predicted positives from pipeline. 3.8% false. 


First selection, based on similarity: Only select molecules with larger similarity with positives than with negatives

In [47]:
selection = pipeline_positives_mol_copy[pipeline_positives_mol_copy['SIM_WITH_POSITIVE_SET'] > pipeline_positives_mol_copy['SIM_WITH_NEGATIVE_SET']]
false_positives = selection[selection['ALDH1_inhibition']==0]
print("{:d} predicted positives after first selection. {:.1f}% false positive. ".format(len(selection),100*len(false_positives)/len(selection)))

52 predicted positives after first selection. 1.9% false positive. 


Second selection: Sort the selection by similarity with positives and pick the first 100 molecules

In [48]:
selection_copy = selection.copy()
selection_copy = selection_copy.sort_values(by=['SIM_WITH_POSITIVE_SET'], ascending=False);
selection_copy = selection_copy.iloc[0:100]
false_positives = selection_copy[selection_copy['ALDH1_inhibition']==0]
print("{:d} predicted positives after second selection. {:.1f}% false positive. ".format(len(selection_copy),100*len(false_positives)/len(selection_copy)))

52 predicted positives after second selection. 1.9% false positive. 


# 4 Do the trick on unknown molecules

Load all untested molecules

In [49]:
unknown_molecules = pd.read_csv(os.path.join(os.getcwd(), PATH_UNTESTED_MOLECULES))
PandasTools.AddMoleculeColumnToFrame(unknown_molecules, smilesCol='SMILES')
unknown_molecules.info()

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'list'

Add all molecular descriptor and calculate them to a pd DataFrame dataset_descriptors.
This time we only used the selected labels to avoid long computational time.

In [ ]:
#descriptors_labels = [n[0] for n in Descriptors._descList[:]]
good_descriptors_labels = ['MaxAbsEStateIndex',
 'MinAbsEStateIndex',
 'MinEStateIndex',
 'BCUT2D_CHGHI',
 'BCUT2D_CHGLO',
 'BCUT2D_LOGPLOW',
 'BCUT2D_MRLOW',
 'SMR_VSA1',
 'SMR_VSA5',
 'SlogP_VSA2',
 'EState_VSA10',
 'VSA_EState2',
 'FractionCSP3',
 'NumAliphaticRings']

In [ ]:
calc = MoleculeDescriptors.MolecularDescriptorCalculator(good_descriptors_labels)
rdkit_desc_unknown_molecules = [calc.CalcDescriptors(m) for m in unknown_molecules["ROMol"]]
dataset_descriptors_unknown_molecules = pd.DataFrame(rdkit_desc_unknown_molecules, columns=[good_descriptors_labels])

Instantiate test(unknown molecules) and train(known molecules) set

In [ ]:
X_train = dataset_descriptors
y_train = dataset_molecules['ALDH1_inhibition']
X_test = dataset_descriptors_unknown_molecules
r = X_test.shape[0]*[0]
y_test = pd.Series(r, copy=False)

### Introducing a Pipeline with minmax scaler and Desciscion tree.

In [ ]:
scaler = pp.MinMaxScaler()
pca = PCA()
dtree = tree.DecisionTreeClassifier(max_features=None, max_depth=None, max_leaf_nodes=None)

pipeline = pl.Pipeline(steps=[("sc", scaler), ("pca", pca), ("dtreeCLF", dtree)])
cross_score = cross_val_score(pipeline, X_train, y_train, cv=10)
print("Cross score:") 
print(cross_score)
pipeline.fit(X_train, y_train)
print("score:")
print(pipeline.score(X_test, y_test))

Cross score:
[0.575 0.635 0.54  0.52  0.55  0.76  0.795 0.855 0.855 0.795]
score:
0.5979


### Creating a physical tree

In [ ]:
#dot_data = tree.export_graphviz(dtree, out_file=None, feature_names=dataset_descriptors.columns, class_names=['1', '0'])
#graph = graphviz.Source(dot_data) 
#graph.render("descision_tree") 

### Prediction

In [ ]:
y_true = y_test
y_pred = pipeline.predict(X_test)

This code calculates further on X_test
A selection of 100 molecules is made, based on the selection from pipeline + further reduction by similarity analysis of the selected molecules with the known ALDH1-positives (and negatives)
Can be appended to Constantijns pipeline.
Als er problemen zijn laat maar weten.

## Further processing of the selection

Create new datafame from X_test. Append values for 'ALDH1_inhibition' and 'predicted'

In [ ]:
pipeline_results = X_test.copy()
pipeline_results['predicted'] = y_pred
pipeline_results.head()
pipeline_results[pipeline_results['predicted'].values != 0]

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPLOW,BCUT2D_MRLOW,SMR_VSA1,SMR_VSA5,SlogP_VSA2,EState_VSA10,VSA_EState2,FractionCSP3,NumAliphaticRings,predicted
0,12.089288,0.112255,-0.251126,2.080331,-2.078497,-2.257675,-0.126034,9.589074,51.993423,17.526044,9.589074,24.017196,0.347826,0,1
1,12.797328,0.238255,-3.572690,2.332267,-2.298232,-2.409391,-0.152691,22.743734,31.081686,50.840649,18.006871,26.898653,0.391304,2,1
2,12.973159,0.116476,-0.893827,2.418534,-2.162707,-2.347503,-0.383654,23.639663,12.462662,29.889309,24.093808,36.375093,0.176471,1,1
3,11.709157,0.004405,-0.628881,2.099290,-2.057211,-2.757900,-0.724720,24.448012,20.268296,23.335118,14.908855,26.672699,0.222222,0,1
4,12.580327,0.039856,-4.469682,2.318223,-2.297318,-2.329215,-0.137383,17.965782,19.017942,23.980671,17.965782,16.768607,0.384615,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9977,11.837918,0.121939,-0.121939,2.047350,-2.051538,-2.226215,0.095014,9.531400,6.544756,18.000956,4.794537,15.974941,0.142857,0,1
9982,12.079166,0.093121,-0.525435,2.180273,-2.102721,-2.151036,-0.385361,14.454711,19.003911,33.661099,14.908855,30.804356,0.266667,0,1
9983,12.286560,0.009186,-3.746215,2.399575,-2.149621,-2.333638,0.085681,17.949197,4.895483,31.781831,13.212334,12.210482,0.133333,1,1
9991,12.705510,0.134945,-0.334667,2.273012,-2.338748,-2.503361,0.056719,23.799663,6.923737,68.805742,9.589074,27.706440,0.500000,1,1


Select only the predicted positives

In [ ]:
pipeline_positives = pipeline_results[pipeline_results['predicted'].values == 1]
print("{} positives from pipeline".format(pipeline_positives.shape[0]))

4021 positives from pipeline


## Further Selection with Tanamoto similarity

Load all known molecules in frame. Now for the whole set of molecules

In [ ]:
from rdkit import DataStructs
tested_molecules = dataset_molecules.copy()
PandasTools.AddMoleculeColumnToFrame(tested_molecules, smilesCol='SMILES')

Divide molecules in Inhibitors and non-Inhibitors

In [ ]:
tested_molecules_pos = tested_molecules[tested_molecules['ALDH1_inhibition'] == 1]
tested_molecules_neg = tested_molecules[tested_molecules['ALDH1_inhibition'] == 0]
print("{}% inhibitors present in known molecules".format(100*len(tested_molecules_pos)/len(tested_molecules)))

30.0% inhibitors present in known molecules


Convert both sets to Morgan Fingerprints (to compare with)

In [ ]:
from rdkit.Chem import AllChem
bulk_pos = [AllChem.GetMorganFingerprintAsBitVect(x,2) for x in tested_molecules_pos['ROMol']]
bulk_neg = [AllChem.GetMorganFingerprintAsBitVect(x,2) for x in tested_molecules_neg['ROMol']]
len_pos = len(bulk_pos); len_neg = len(bulk_neg)

Get indices from pipeline prediction and select these indices from dataset_molecules.
This must be done to retreive SMILES again, since these are needed for the comparison in the for loop

In [ ]:
index_pipeline_positives = pipeline_positives.index
pipeline_positives_mol = unknown_molecules.iloc[index_pipeline_positives]
print("{} positive molecules from pipeline".format(len(pipeline_positives_mol)))

4021 positive molecules from pipeline


Calculate similarity with tested positives/negatives for every mol from pipeline_positives

In [ ]:
pipeline_positives_mol_copy = pipeline_positives_mol.copy(deep=False)
# Loop all rows from test set
for index, row in pipeline_positives_mol.iterrows():     
    test_smiles = row['SMILES']
    # Make allchem MolFrom the SMILES and convert it into to GetMorganFingerprintAsBitVec
    test_mol = AllChem.MolFromSmiles(test_smiles)
    ref_fps = AllChem.GetMorganFingerprintAsBitVect(test_mol,2)
    # Calculate similarities with positives and negative
    similarity_pos = [DataStructs.FingerprintSimilarity(ref_fps,x) for x in bulk_pos]
    similarity_neg = [DataStructs.FingerprintSimilarity(ref_fps,x) for x in bulk_neg]
    # Add all positive and negative similarities and normalize
    rel_sum_pos = sum(similarity_pos)/len_pos # divide by sizes to normalize the result
    rel_sum_neg = sum(similarity_neg)/len_neg # divide by sizes to normalize the result
    
    pipeline_positives_mol_copy.loc[index, ['SIM_WITH_POSITIVE_SET']] = rel_sum_pos
    pipeline_positives_mol_copy.loc[index, ['SIM_WITH_NEGATIVE_SET']] = rel_sum_neg   

pipeline_positives_mol_copy.head()

,SMILES,ROMol,SIM_WITH_POSITIVE_SET,SIM_WITH_NEGATIVE_SET
0,CCCCC/C(=N\NC(=O)CCC(=O)NCc1ccccc1)c1ccccc1,<rdkit.Chem.rdchem.Mol object at 0x000001FFAF2...,0.134472,0.119836
1,Cc1ccc(S(=O)(=O)N2CCC(C(=O)OCC(=O)N3CCc4ccccc4...,<rdkit.Chem.rdchem.Mol object at 0x000001FFAF2...,0.154441,0.107367
2,COc1ccc(N2C(=O)CC(Nc3ccc(F)cc3)C2=O)c([N+](=O)...,<rdkit.Chem.rdchem.Mol object at 0x000001FFAF2...,0.125733,0.091493
3,Cc1cc(C)cc(OCC(=O)O[NH+]=C(N)Cc2ccc([N+](=O)[O...,<rdkit.Chem.rdchem.Mol object at 0x000001FFAF2...,0.127386,0.101580
4,NNC(=O)C=C1CCN(c2ncc(C(F)(F)F)cc2Cl)CC1,<rdkit.Chem.rdchem.Mol object at 0x000001FFAF2...,0.106596,0.095824


Only select molecules with larger similarity with positives than with negatives

In [ ]:
selection = pipeline_positives_mol_copy[pipeline_positives_mol_copy['SIM_WITH_POSITIVE_SET'] > pipeline_positives_mol_copy['SIM_WITH_NEGATIVE_SET']]
print("{:d} predicted positives after first selection".format(len(selection)))

3615 predicted positives after first selection


Sort the selection by similarity with positives and pick the first 100 molecules

In [ ]:
selection_copy = selection.copy()
selection_copy = selection_copy.sort_values(by=['SIM_WITH_POSITIVE_SET'], ascending=False);
selection_copy = selection_copy.iloc[0:100]
print("{:d} predicted positives after first selection".format(len(selection_copy)))

100 predicted positives after first selection


In [ ]:
definite_selection = selection_copy
indexes = definite_selection.index

In [ ]:
selected_smiles = definite_selection['SMILES']

In [ ]:
selected_smiles.to_csv('100_selected_ALDH1_inhibitors.csv', index=False)  